# Visualização de um banco de dados

### Alunos: Igor Patrício Michels e Igor Cortes Junqueira

In [1]:
import io
from itertools import count
import pymysql
import getpass
import pandas as pd

import networkx as nx
from networkx import Graph as NXGraph
from networkx.drawing.nx_agraph import graphviz_layout

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import collections

from IPython.display import display, Image

from ipywidgets import interactive, HBox, BoundedIntText

In [2]:
p = getpass.getpass()

 ··········


In [3]:
conn = pymysql.connect(host = 'localhost',
                       user = 'root', 
                       passwd = p,
                       database = 'usa_flights') # e mudar aqui para flights (completo) / usa_flights (amostral de 05-2020)

cur = conn.cursor()

Aqui estamos fazendo algumas querys e criando alguns dataframes de modo a otimizar o processo de criação do grafo. Esse processo agiliza a criação dos grafos por evitar a realização do mesmo a cada novo grafo. Nele estamos criando 4 dataframes: um de companhias, um de fabricantes, um de aeroportos e um de rotas. Os dataframes de companhias e de aeroportos tem o papel de serem auxiliares na criação das querys dos outros dois, os quais são utilizados na confecção dos grafos.

In [11]:
max_companies = 12
max_airports = 30

# query para encontrar as companhias
query = '''
        SELECT Companies.name AS Company,
               COUNT(Aircrafts.N_Number) AS Flights
        FROM Flights
        INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
        INNER JOIN Routes ON Flights.Route_id = Routes.id_route
        INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
        GROUP BY Companies.name
        ORDER BY Flights DESC
        LIMIT {}
        ;
        '''.format(max_companies)
companies = pd.read_sql_query(query, conn)
include = []
for i in range(max_companies):
    include.append("Companies.name = '{}'".format(companies.loc[i, 'Company']))

where_comp = ""
for k in include:
    if k == include[-1]:
        where_comp += k
    else:
        where_comp += k
        where_comp += " OR "

# query para encontrar os fabricantes
query = '''
        SELECT C.Company AS Company,
               M.Name AS Manufacturer,
               COUNT(A.N_Number) AS Aircrafts
        FROM (SELECT Companies.name AS Company,
                     Companies.id_company AS ID,
                     COUNT(Aircrafts.N_Number) AS Flights
              FROM Flights
              INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
              INNER JOIN Routes ON Flights.Route_id = Routes.id_route
              INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
              GROUP BY Companies.name
              ORDER BY Flights DESC
              LIMIT {}) AS C
        INNER JOIN aircrafts AS A ON C.ID = A.Companys_id
        INNER JOIN aircraft_models AS AM ON A.Aircraft_Models_id = AM.id_model
        INNER JOIN manufacturer AS M ON AM.Manufacturer_id = M.id_manufacturer
        GROUP BY Company, Manufacturer
        ORDER BY Aircrafts DESC
        ;
        '''.format(max_companies)

manufacturers = pd.read_sql_query(query, conn)

for fabricante in manufacturers.index:
    if manufacturers.loc[fabricante, 'Manufacturer'] == 'EMBRAER-EMPRESA BRASILEIRA DE' or manufacturers.loc[fabricante, 'Manufacturer'] == 'EMBRAER S A':
        manufacturers.loc[fabricante, 'Manufacturer'] = 'EMBRAER'
    elif manufacturers.loc[fabricante, 'Manufacturer'] == 'MCDONNELL DOUGLAS CORPORATION' or manufacturers.loc[fabricante, 'Manufacturer'] == 'MCDONNELL DOUGLAS AIRCRAFT CO':
        manufacturers.loc[fabricante, 'Manufacturer'] = 'MCDONNELL DOUGLAS'

# query para encontrar os aeroportos
query = '''
        SELECT A.Arrival AS Airport,
               A.Flights + D.Flights AS Total
        FROM (SELECT Routes.Arrival_Airport_id AS Arrival,
                     COUNT(Aircrafts.N_Number) AS Flights
              FROM Flights
              INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
              INNER JOIN Routes ON Flights.Route_id = Routes.id_route
              INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
              WHERE {0}
              GROUP BY Routes.Arrival_Airport_id
              ORDER BY RAND()) AS A
        INNER JOIN (SELECT Routes.Departure_Airport_id AS Departure,
                    COUNT(Aircrafts.N_Number) AS Flights
                    FROM Flights
                    INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
                    INNER JOIN Routes ON Flights.Route_id = Routes.id_route
                    INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
                    WHERE {1}
                    GROUP BY Routes.Departure_Airport_id
                    ORDER BY RAND()) AS D
        ON A.Arrival = D.Departure
        ORDER BY Total DESC
        LIMIT {2}
        ;
        '''.format(where_comp, where_comp, max_airports)

airports = pd.read_sql_query(query, conn)

airports1 = []
airports2 = []
for i in range(max_airports):
    airports1.append("Routes.Departure_Airport_id = '{}'".format(airports.loc[i, 'Airport']))
    airports2.append("Routes.Arrival_Airport_id = '{}'".format(airports.loc[i, 'Airport']))

where_airp1 = ""
for k in airports1:
    if k == airports1[-1]:
        where_airp1 += k
    else:
        where_airp1 += k
        where_airp1 += " OR "

where_airp2 = ""
for k in airports2:
    if k == airports2[-1]:
        where_airp2 += k
    else:
        where_airp2 += k
        where_airp2 += " OR "

# query para encontrar as rotas
query = '''
        SELECT Companies.name AS Company,
               Routes.Departure_Airport_id AS Departure,
               Routes.Arrival_Airport_id AS Arrival,
               COUNT(Aircrafts.N_Number) AS Flights
        FROM Flights
        INNER JOIN Aircrafts ON Aircrafts.n_number = Flights.Aircrafts_id
        INNER JOIN Routes ON Flights.Route_id = Routes.id_route
        INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
        WHERE ({0}) AND ({1}) AND ({2})
        GROUP BY Companies.name, Routes.Departure_Airport_id, Routes.Arrival_Airport_id
        ORDER BY Flights DESC
        ;
        '''.format(where_comp, where_airp1, where_airp2)

routes = pd.read_sql_query(query, conn)

DatabaseError: Execution failed on sql '
        SELECT C.Company AS Company,
               M.Name AS Manufacturer,
               COUNT(A.N_Number) AS Aircrafts
        FROM (SELECT Companies.name AS Company,
                     Companies.id_company AS ID,
                     COUNT(Aircrafts.N_Number) AS Flights
              FROM Flights
              INNER JOIN Aircrafts ON Aircrafts.N_Number = Flights.Aircrafts_id
              INNER JOIN Routes ON Flights.Route_id = Routes.id_route
              INNER JOIN Companies ON Companies.id_company = Aircrafts.Companys_id
              GROUP BY Companies.name
              ORDER BY Flights DESC
              LIMIT 12) AS C
        INNER JOIN Aircrafts AS A ON C.ID = A.Companys_id
        INNER JOIN Aircraft_Models AS AM ON A.Aircraft_Models_id = AM.id_model
        INNER JOIN Manufacturer AS M ON AM.Manufacturer_id = M.id_manufacturer
        GROUP BY Company, Manufacturer
        ORDER BY Aircrafts DESC
        ;
        ': (1055, "Exp

Abaixo temos a função interativa de criação de grafos.

In [4]:
info = '''
No grafo abaixo temos em 'azure' os aeroportos, em 'lightskyblue' as companhias e em 'turquoise' as empresas fabricantes de \naeronaves.

A interação entre aeroportos se dá através de uma aresta que significa que existe uma rota entre dois aeroportos.

Já a interação entre companhias e aeroportos se dá por uma arestas que representa que a companhia opera naquele aeroporto.

Por fim, a interação entre companhias e fabricantes se dá por uma aresta que representa que um fabricante fabricou aeronaves \npara aquela companhia.

Optamos por limitar a quantidade de companhias a 12, embora possamos mudar esse limite na primeira linha da célula anterior \ne a quantidade de aeroportos a 30, com a mesma ressalva. Já o número de fabricantes limitamos a 5 com o intuito de não \npoluir muito o grafo.

Além disso, podemos deixar a quantidade de aeroportos e de fabricantes igual a 0, o que possibilita fazer análises de quem \nfabrica aeronaves para cada companhia ou que companhia opera em que aeroporto.

'''

def graph(n_companies, n_airports, n_manufacturers):
    if n_airports == 0 and n_manufacturers == 0:
        text = '''Plotando a(s) {} maior(es) companhia(s).
        '''.format(n_companies)
    elif n_manufacturers == 0:
        text = '''Plotando a(s) {} companhia(s) junto do(s) {} aeroporto(s) mais movimentados.
        '''.format(n_companies, n_airports)
    elif n_airports == 0:
        text = '''Plotando a(s) {} companhia(s) junto do(s) {} maior(es) fabricante(s).
        '''.format(n_companies, n_manufacturers)
    else:
        text = '''Plotando a(s) {} companhia(s) junto do(s) {} aeroporto(s) mais movimentados e do(s) {} maior(es) fabricante(s)
        '''.format(n_companies, n_airports, n_manufacturers)
    
    print(text)
    print()
    print(info)
    
    comp_add = 0
    airp_add = 0
    fab_add = 0

    G = nx.DiGraph()
    for rota in routes.index:
        # colocando as companhias e aeroportos no grafo
        if routes.loc[rota, 'Departure'] not in G.nodes() and routes.loc[rota, 'Arrival'] not in G.nodes() and airp_add <= n_airports - 2:
            G.add_node(routes.loc[rota, 'Departure'], color = 'azure')
            G.add_node(routes.loc[rota, 'Arrival'], color = 'azure')
            airp_add += 2
        elif routes.loc[rota, 'Departure'] not in G.nodes() and airp_add < n_airports:
            G.add_node(routes.loc[rota, 'Departure'], color = 'azure')
            airp_add += 1
        elif routes.loc[rota, 'Arrival'] not in G.nodes() and airp_add < n_airports:
            G.add_node(routes.loc[rota, 'Arrival'], color = 'azure')
            airp_add += 1

        if routes.loc[rota, 'Company'] not in G.nodes() and comp_add < n_companies:
            G.add_node(routes.loc[rota, 'Company'], color = 'lightskyblue')
            comp_add += 1

        if routes.loc[rota, 'Departure'] in G.nodes() and routes.loc[rota, 'Arrival'] in G.nodes() and routes.loc[rota, 'Company'] in G.nodes():
            if routes.loc[rota, 'Departure'] not in G.nodes():
                G.add_node(routes.loc[rota, 'Departure'], color = 'azure')
                airp_add += 1
            if routes.loc[rota, 'Arrival'] not in G.nodes():
                G.add_node(routes.loc[rota, 'Arrival'], color = 'azure')
                airp_add += 1
            G.add_edge(routes.loc[rota, 'Departure'], routes.loc[rota, 'Arrival'],
                       weight = routes.loc[rota, 'Flights'],
                       color = 'steelblue',
                       relation = 'tem rota para')
            if (routes.loc[rota, 'Company'], routes.loc[rota, 'Departure']) in G.edges():
                G.edges()[(routes.loc[rota, 'Company'], routes.loc[rota, 'Departure'])]['weight'] += routes.loc[rota, 'Flights']
            else:
                G.add_edge(routes.loc[rota, 'Company'], routes.loc[rota, 'Departure'],
                           weight = routes.loc[rota, 'Flights'],
                           color = 'turquoise',
                           relation = 'opera em')
            if (routes.loc[rota, 'Company'], routes.loc[rota, 'Arrival']) in G.edges():
                G.edges()[(routes.loc[rota, 'Company'], routes.loc[rota, 'Arrival'])]['weight'] += routes.loc[rota, 'Flights']
            else:
                G.add_edge(routes.loc[rota, 'Company'], routes.loc[rota, 'Arrival'],
                           weight = routes.loc[rota, 'Flights'],
                           color = 'turquoise',
                           relation = 'opera em')

    for fabricante in manufacturers.index:
        # colocando os fabricantes no grafo
        if fab_add < n_manufacturers and manufacturers.loc[fabricante, 'Company'] in G.nodes():
            if manufacturers.loc[fabricante, 'Manufacturer'] not in G.nodes():
                G.add_node(manufacturers.loc[fabricante, 'Manufacturer'], color = 'cadetblue')
                fab_add += 1
            G.add_edge(manufacturers.loc[fabricante, 'Manufacturer'], manufacturers.loc[fabricante, 'Company'],
                       weight = manufacturers.loc[fabricante, 'Aircrafts'],
                       color = 'turquoise',
                       relation = 'fabricou para')
    
    # plotando
    fig, ax = plt.subplots(figsize=(20,20))
    pos = nx.spring_layout(G, scale=1)
    nx.draw_networkx_edge_labels(G, 
                                 pos, 
                                 edge_labels = nx.get_edge_attributes(G,'relation'),
                                 label_pos = 0.5, 
                                 font_size = 9, 
                                 font_color = 'red', 
                                 font_family = 'sans-serif', 
                                 font_weight = 'normal', 
                                 alpha = 1.0, 
                                 bbox = None, 
                                 ax = ax, 
                                 rotate = True)

    nx.draw_networkx(G,
                     pos = pos,
                     ax = ax,
                     node_color = [nx.get_node_attributes(G,'color')[g] for g in G.nodes()],
                     edge_color = [nx.get_edge_attributes(G,'color')[g] for g in G.edges()],)

    plt.show()

# variáveis do interact
n_companies = BoundedIntText(value = 6,
                             min = 1,
                             max = max_companies,
                             step = 1,
                             description = 'Companhias:',
                             disabled = False)
n_airports = BoundedIntText(value = 10,
                            min = 0,
                            max = max_airports,
                            step = 1,
                            description = 'Aeroportos:',
                            disabled = False)
n_manufacturers = BoundedIntText(value = 3,
                                 min = 0,
                                 max = 5,
                                 step = 1,
                                 description = 'Fabricantes:',
                                 disabled = False)

# execução do grafo
ip = interactive(graph,
                 n_companies = n_companies,
                 n_airports = n_airports,
                 n_manufacturers = n_manufacturers)
print("Escolha quantas companhias e quantos aeroportos plotar. Essas escolhas são feitas de acordo com o volume de voo")
display(HBox(ip.children[0:3]))
print("Para a primeira plotagem faça alguma alteração nas seleções acima.")
display(ip.children[-1])

Escolha quantas companhias e quantos aeroportos plotar. Essas escolhas são feitas de acordo com o volume de voo


Para a primeira plotagem faça alguma alteração nas seleções acima.


Output()